# Ranker that can takes into accound different features

# Set up

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import os
import sys
from typing import List

import dill
import lightning as L
import numpy as np
import pandas as pd
import torch
from dotenv import load_dotenv
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from mlflow.models.signature import infer_signature
from pydantic import BaseModel
from torch.utils.data import DataLoader

import mlflow

load_dotenv()

sys.path.insert(0, "..")

from src.ann import AnnIndex
from src.data_prep_utils import chunk_transform
from src.dataset import UserItemBinaryDFDataset
from src.id_mapper import IDMapper
from src.ranker.inference import RankerInferenceWrapper
from src.ranker.model import Ranker
from src.ranker.trainer import LitRanker
from src.viz import blueq_colors

# Controller

In [3]:
max_epochs = 100

In [4]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "RecSys MVP - Ranker"
    run_name: str = "027-refit-with-features-and-title"
    notebook_persist_dp: str = None
    random_seed: int = 41
    device: str = None

    # Feature flagS
    use_sbert_features: bool = True

    item_metadata_pipeline_fp: str = "../data/item_metadata_pipeline.dill"
    qdrant_url: str = None
    qdrant_collection_name: str = "item_desc_sbert"

    max_epochs: int = max_epochs
    batch_size: int = 128
    tfm_chunk_size: int = 10000
    neg_to_pos_ratio: int = 1

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    item_feature_cols: List[str] = [
        "main_category",
        "categories",
        "price",
        "parent_asin_rating_cnt_365d",
        "parent_asin_rating_avg_prev_rating_365d",
        "parent_asin_rating_cnt_90d",
        "parent_asin_rating_avg_prev_rating_90d",
        "parent_asin_rating_cnt_30d",
        "parent_asin_rating_avg_prev_rating_30d",
        "parent_asin_rating_cnt_7d",
        "parent_asin_rating_avg_prev_rating_7d",
    ]

    top_K: int = 100
    top_k: int = 10

    embedding_dim: int = 128
    dropout: float = 0.3
    early_stopping_patience: int = 5
    learning_rate: float = 0.0003
    l2_reg: float = 1e-4

    mlf_item2vec_model_name: str = "item2vec"
    mlf_model_name: str = "ranker"
    min_roc_auc: float = 0.7

    best_checkpoint_path: str = None

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)

        if not (qdrant_host := os.getenv("QDRANT_HOST")):
            raise Exception(f"Environment variable QDRANT_HOST is not set.")

        qdrant_port = os.getenv("QDRANT_PORT")
        self.qdrant_url = f"{qdrant_host}:{qdrant_port}"

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-10-28 11:39:29.221 | INFO     | __main__:init:72 - Setting up MLflow experiment RecSys MVP - Ranker - run 027-refit-with-features-and-title...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "RecSys MVP - Ranker",
  "run_name": "027-refit-with-features-and-title",
  "notebook_persist_dp": "/Users/dvq/frostmourne/recsys-mvp/notebooks/data/027-refit-with-features-and-title",
  "random_seed": 41,
  "device": null,
  "use_sbert_features": true,
  "item_metadata_pipeline_fp": "../data/item_metadata_pipeline.dill",
  "qdrant_url": "localhost:6333",
  "qdrant_collection_name": "item_desc_sbert",
  "max_epochs": 100,
  "batch_size": 128,
  "tfm_chunk_size": 10000,
  "neg_to_pos_ratio": 1,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "item_feature_cols": [
    "main_category",
    "categories",
    "price",
    "parent_asin_rating_cnt_365d",
    "parent_asin_rating_avg_prev_rating_365d",
    "parent_asin_rating_cnt_90d",
    "parent_asin_rating_avg_prev_rating_90d",
    "parent_asin_rating_cnt_30d",
    "parent_asin_rating_avg_prev_rating_30d",
  

# Implement

In [5]:
def init_model(
    n_users, n_items, embedding_dim, item_feature_size, dropout, item_embedding=None
):
    model = Ranker(
        n_users,
        n_items,
        embedding_dim,
        item_feature_size=item_feature_size,
        dropout=dropout,
        item_embedding=item_embedding,
    )
    return model

## Load pretrained Item2Vec embeddings

In [6]:
mlf_client = mlflow.MlflowClient()
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{args.mlf_item2vec_model_name}@champion"
)
skipgram_model = model.unwrap_python_model().model
embedding_0 = skipgram_model.embeddings(torch.tensor(0))
embedding_dim = embedding_0.size()[0]
id_mapping = model.unwrap_python_model().id_mapping
pretrained_item_embedding = skipgram_model.embeddings

In [7]:
assert (
    pretrained_item_embedding.embedding_dim == args.embedding_dim
), "Mismatch pretrained item_embedding dimension"

## Load vectorized item features

In [8]:
with open(args.item_metadata_pipeline_fp, "rb") as f:
    item_metadata_pipeline = dill.load(f)

## Load ANN Index

In [9]:
ann_index = AnnIndex(args.qdrant_url, args.qdrant_collection_name)

In [10]:
vector = ann_index.get_vector_by_ids([0])[0]
embedding_dim = vector.shape[0]

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
sbert_embedding_dim = vector.shape[0]
neighbors = ann_index.get_neighbors_by_ids([0])
neighbors

  0%|          | 0/1 [00:00<?, ?it/s]

[ScoredPoint(id=0, version=0, score=0.0, payload={'parent_asin': '0375869026', 'title': 'Wonder'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1916, version=59, score=0.3048898, payload={'parent_asin': 'B005GFPZYK', 'title': 'American Sniper: The Autobiography of the Most Lethal Sniper in U.S. Military History'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=166, version=5, score=0.35464486, payload={'parent_asin': 'B00005OARM', 'title': 'Golden Sun'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3845, version=120, score=0.3593233, payload={'parent_asin': 'B0794W1LWG', 'title': 'Life is Strange: Before The Storm Limited Edition - Xbox One'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3896, version=121, score=0.3675484, payload={'parent_asin': 'B07CD6F5PX', 'title': 'Dragon Quest Xi: Echoes of An Elusive Age - PlayStation 4'}, vector=None, shard_key=None, order_value=None)]

# Test implementation

In [12]:
embedding_dim = 8
batch_size = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 4, 5, 3, 0]
item_sequences = [
    [-1, -1, 2, 3],
    [-1, -1, 2, 3],
    [-1, -1, 1, 3],
    [-1, -1, 2, 1],
    [-1, -1, 2, 1],
]
main_category = [
    "All Electronics",
    "Video Games",
    "All Electronics",
    "Video Games",
    "Unknown",
]
categories = [[], ["Headsets"], ["Video Games"], [], ["blah blah"]]
title = ["World of Warcraft", "DotA 2", "Diablo IV", "Football Manager 2024", "Unknown"]
description = [[], [], ["Video games blah blah"], [], ["blah blah"]]
price = ["from 14.99", "14.99", "price: 9.99", "20 dollars", "None"]
parent_asin_rating_cnt_365d = [0, 1, 2, 3, 4]
parent_asin_rating_avg_prev_rating_365d = [4.0, 3.5, 4.5, 5.0, 2.0]
parent_asin_rating_cnt_90d = [0, 1, 2, 3, 4]
parent_asin_rating_avg_prev_rating_90d = [4.0, 3.5, 4.5, 5.0, 2.0]
parent_asin_rating_cnt_30d = [0, 1, 2, 3, 4]
parent_asin_rating_avg_prev_rating_30d = [4.0, 3.5, 4.5, 5.0, 2.0]
parent_asin_rating_cnt_7d = [0, 1, 2, 3, 4]
parent_asin_rating_avg_prev_rating_7d = [4.0, 3.5, 4.5, 5.0, 2.0]

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
        "item_sequence": item_sequences,
        "main_category": main_category,
        "title": title,
        "description": description,
        "categories": categories,
        "price": price,
        "parent_asin_rating_cnt_365d": parent_asin_rating_cnt_365d,
        "parent_asin_rating_avg_prev_rating_365d": parent_asin_rating_avg_prev_rating_365d,
        "parent_asin_rating_cnt_90d": parent_asin_rating_cnt_90d,
        "parent_asin_rating_avg_prev_rating_90d": parent_asin_rating_avg_prev_rating_90d,
        "parent_asin_rating_cnt_30d": parent_asin_rating_cnt_30d,
        "parent_asin_rating_avg_prev_rating_30d": parent_asin_rating_avg_prev_rating_30d,
        "parent_asin_rating_cnt_7d": parent_asin_rating_cnt_7d,
        "parent_asin_rating_avg_prev_rating_7d": parent_asin_rating_avg_prev_rating_7d,
    }
)
train_item_features = item_metadata_pipeline.transform(train_df).astype(np.float32)
if args.use_sbert_features:
    sbert_vectors = ann_index.get_vector_by_ids(
        train_df["item_indice"].values.tolist()
    ).astype(np.float32)
    train_item_features = np.hstack([train_item_features, sbert_vectors])

n_users = len(set(user_indices))
n_items = len(set(item_indices))
item_feature_size = train_item_features.shape[1]

model = init_model(n_users, n_items, embedding_dim, item_feature_size, args.dropout)

# Example forward pass
model.eval()
users = torch.tensor(user_indices)
items = torch.tensor(item_indices)
item_sequences = torch.tensor(item_sequences)
item_features = torch.tensor(train_item_features)
predictions = model.predict(users, item_sequences, item_features, items)
print(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[0.4601],
        [0.4034],
        [0.5219],
        [0.5240],
        [0.4168]], grad_fn=<SigmoidBackward0>)


In [13]:
rating_dataset = UserItemBinaryDFDataset(
    train_df,
    "user_indice",
    "item_indice",
    args.rating_col,
    args.timestamp_col,
    item_feature=train_item_features,
)

train_loader = DataLoader(rating_dataset, batch_size=batch_size, shuffle=False)

In [14]:
for batch_input in train_loader:
    print(batch_input)

{'user': tensor([0, 0]), 'item': tensor([0, 1]), 'rating': tensor([0., 1.]), 'item_sequence': tensor([[-1, -1,  2,  3],
        [-1, -1,  2,  3]]), 'item_feature': tensor([[-1.4698e-02,  5.6424e+00, -1.4698e-02,  ...,  2.2739e-02,
         -2.3894e-02,  1.1594e-03],
        [-1.4698e-02, -1.7723e-01, -1.4698e-02,  ..., -2.3171e-03,
         -2.9986e-02,  9.9219e-03]])}
{'user': tensor([1, 2]), 'item': tensor([2, 3]), 'rating': tensor([1., 1.]), 'item_sequence': tensor([[-1, -1,  1,  3],
        [-1, -1,  2,  1]]), 'item_feature': tensor([[-1.4698e-02,  5.6424e+00, -1.4698e-02,  ...,  2.1281e-03,
         -3.6321e-02,  4.1318e-04],
        [-1.4698e-02, -1.7723e-01, -1.4698e-02,  ..., -4.9983e-03,
         -5.8112e-02,  4.0446e-03]])}
{'user': tensor([2]), 'item': tensor([4]), 'rating': tensor([0.]), 'item_sequence': tensor([[-1, -1,  2,  1]]), 'item_feature': tensor([[-1.4698e-02, -1.7723e-01, -1.4698e-02, -2.5463e-02, -1.4698e-02,
         -1.4698e-02, -2.0788e-02, -8.2101e-02, -2.872

In [15]:
# Prepare all item features for recommendation
all_items_df = train_df.drop_duplicates(subset=["item_indice"])
all_items_indices = all_items_df["item_indice"].values
all_items_features = item_metadata_pipeline.transform(all_items_df).astype(np.float32)
if args.use_sbert_features:
    all_sbert_vectors = ann_index.get_vector_by_ids(all_items_indices.tolist()).astype(
        np.float32
    )
    all_items_features = np.hstack([all_items_features, all_sbert_vectors])

lit_model = LitRanker(
    model,
    log_dir=args.notebook_persist_dp,
    all_items_indices=all_items_indices,
    all_items_features=all_items_features,
)

# train model
trainer = L.Trainer(
    default_root_dir=f"{args.notebook_persist_dp}/test",
    max_epochs=2,
    accelerator=args.device if args.device else "auto",
)
trainer.fit(
    model=lit_model, train_dataloaders=train_loader, val_dataloaders=train_loader
)

  0%|          | 0/1 [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params | Mode
----------------------------------------
0 | model | Ranker | 8.2 K  | eval
----------------------------------------
8.2 K     Trainable params
0         Non-trainable params
8.2 K     Total params
0.033     Total estimated model params size (MB)
0         Modules in train mode
14        Modules in eval mode


Sanity Checking: |                                                                                            …

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.
/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.
/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=2` reached.
2024-10-28 11:39:31.573 | INFO     | src.ranker.trainer:on_fit_end:158 - Logging classification metrics...


In [16]:
# After fitting
model.eval()
predictions = model.predict(users, item_sequences, item_features, items)
print(predictions)

tensor([[0.4521],
        [0.4149],
        [0.5390],
        [0.5351],
        [0.4201]], grad_fn=<SigmoidBackward0>)


In [17]:
# Get the last row of each item as input for recommendations (containing the most updated item_sequence)
to_rec_df = train_df.sort_values(args.timestamp_col, ascending=False).drop_duplicates(
    subset=["user_indice"]
)
recommendations = model.recommend(
    torch.tensor(to_rec_df["user_indice"].values.tolist()),
    torch.tensor(to_rec_df["item_sequence"].values.tolist()),
    torch.tensor(lit_model.all_items_features),
    torch.tensor(lit_model.all_items_indices),
    k=2,
    batch_size=4,
)
recommendations

Generating recommendations:   0%|          | 0/1 [00:00<?, ?it/s]

{'user_indice': [2, 2, 1, 1, 0, 0],
 'recommendation': [2, 3, 3, 2, 3, 2],
 'score': [0.5518050789833069,
  0.5350745916366577,
  0.5438730716705322,
  0.5389871597290039,
  0.4857848286628723,
  0.4834492802619934]}

# Prep data

In [18]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm_fp = "../data/idm.json"
idm = IDMapper().load(idm_fp)

assert (
    train_df[args.user_col].map(lambda s: idm.get_user_index(s))
    != train_df["user_indice"]
).sum() == 0, "Mismatch IDM"
assert (
    val_df[args.user_col].map(lambda s: idm.get_user_index(s)) != val_df["user_indice"]
).sum() == 0, "Mismatch IDM"

In [19]:
train_df

,user_id,parent_asin,rating,timestamp,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,parent_asin_rating_avg_prev_rating_30d,...,item_indice,main_category,title,description,categories,price,user_rating_cnt_90d,user_rating_avg_prev_rating_90d,user_rating_list_10_recent_asin,item_sequence
0,AG57LGJFCNNQJ6P6ABQAVUKXDUDA,B0015AARJI,0.0,2016-01-12 11:59:11.000,76.0,4.592105,10.0,4.3,3.0,5.0,...,660,Video Games,PlayStation 3 Dualshock 3 Wireless Controller ...,"[Amazon.com, The Dualshock 3 wireless controll...","[Video Games, Legacy Systems, PlayStation Syst...",49.99,2,5.000000,B00J00BLRM,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2662]"
1,AHWG4EGOV5ZDKPETL56MAYGPLJRQ,B0BMGHMP23,0.0,2016-04-18 19:26:20.000,NaN,NaN,NaN,NaN,NaN,NaN,...,4568,Computers,Logitech G502 Lightspeed Wireless Gaming Mouse...,[G502 is the best gaming mouse from Logitech G...,"[Video Games, PC, Accessories, Gaming Mice]",87.95,3,5.000000,"B00YOGZFCO,B00KWFCSB2,B00L3LQ1FI,B0151K6J9Y,B0...","[3028, 2742, 2755, 3159, 3101, 3036, 3051, 313..."
2,AH5PTZ2U74OZ3HT6QVUWM4CV6OVQ,B009AP23NI,0.0,2016-02-10 18:45:08.000,9.0,4.666667,0.0,NaN,0.0,NaN,...,2219,Video Games,Nintendo Wii U Pro U Controller (Japanese Vers...,[Wii U PRO controller (black) (WUP-A-RSKA)],"[Video Games, Legacy Systems, Nintendo Systems...",43.99,8,4.428571,"B0199OXR0W,B00EVPR4FY,B00B7ELWAU,B00UH9DN58,B0...","[-1, -1, 3234, 2508, 2318, 2964, 1258, 2439, 4..."
3,AFC5XTCF5D7J3NSDITB2Z26XWWYA,B001E8WQUY,5.0,2019-05-01 21:22:39.265,0.0,NaN,0.0,NaN,0.0,NaN,...,724,Video Games,Rock Band 2 - Nintendo Wii (Game only),"[Product description, Rock Band 2 lets you and...","[Video Games, Legacy Systems, Nintendo Systems...",28.49,1,NaN,"B006HZA6VK,B0BN2FNKLM,B0086VPUHI,B0040UAYI4,B0...","[1987, 4569, 2114, 1606, 2159, 2279, 2447, 441..."
4,AF7LJQOIWF3Y3YD7SGOJ34MA5JPA,B001E8WQKY,5.0,2015-01-09 12:53:25.000,16.0,4.375000,8.0,4.5,4.0,4.5,...,722,Video Games,Resident Evil 5 - Xbox 360,[],"[Video Games, Legacy Systems, Xbox Systems, Xb...",29.88,3,5.000000,"B00A2ML6XG,B003VUO6LU","[-1, -1, -1, -1, -1, -1, -1, -1, 2261, 1579]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328591,AG4RATLNVLOKZCPXN67HKOAK65CA,B078FBVJMB,0.0,2015-10-31 18:25:09.000,NaN,NaN,NaN,NaN,NaN,NaN,...,3829,Video Games,A Way Out – PC Origin [Online Game Code],[From the creators of Brothers - A Tale of Two...,"[Video Games, PC, Games]",5.99,1,NaN,B00TFVD688,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2951]"
328592,AFBXO3BFWBJX6QS5NW73O37IXF2A,B0771ZXXV6,0.0,2011-03-08 02:06:38.000,NaN,NaN,NaN,NaN,NaN,NaN,...,3806,Video Games,Nintendo Joy-Con (R) - Neon Red - Nintendo Switch,[To be determined],"[Video Games, Nintendo Switch, Accessories, Co...",None,3,4.000000,"B003JVCA9Q,B0029NZ4HA","[-1, -1, -1, -1, -1, -1, -1, -1, 1488, 1199]"
328593,AHVANA5GZNJ45UABPXWZNAF4ECBQ,B00BBF6MO6,0.0,2015-02-15 05:31:04.000,3.0,4.666667,0.0,NaN,0.0,NaN,...,2327,Video Games,Killer is Dead - Xbox 360,[Killer Is Dead is the latest title from the d...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",39.82,1,NaN,"B002L93F0A,B002KJ02ZC,B001H4NMNA","[-1, -1, -1, -1, -1, -1, -1, 1377, 1374, 1092]"
328594,AHAVA5VKMJ3OMOLGDZ3W45CKXEWA,B00KTORA0K,5.0,2019-05-25 04:03:51.505,3.0,4.666667,1.0,5.0,1.0,5.0,...,2726,Video Games,Just Dance 2015 - Wii,[With more than 50 million copies of Just Danc...,"[Video Games, Legacy Systems, Nintendo Systems...",33.0,2,5.000000,"B004AYCNR0,B007NUQICE,B000TYQL1O,B000SEU92W,B0...","[-1, -1, -1, 1657, 2074, 593, 583, 3715, 3448,..."


In [20]:
user_indices = train_df["user_indice"].unique()
item_indices = train_df["item_indice"].unique()
if args.use_sbert_features:
    all_sbert_vectors = ann_index.get_vector_by_ids(
        item_indices.tolist(), chunk_size=1000
    ).astype(np.float32)

train_item_features = chunk_transform(
    train_df, item_metadata_pipeline, chunk_size=args.tfm_chunk_size
)
train_item_features = train_item_features.astype(np.float32)

val_item_features = chunk_transform(
    val_df, item_metadata_pipeline, chunk_size=args.tfm_chunk_size
)
val_item_features = val_item_features.astype(np.float32)

if args.use_sbert_features:
    train_sbert_vectors = all_sbert_vectors[train_df["item_indice"].values]
    train_item_features = np.hstack([train_item_features, train_sbert_vectors])
    val_sbert_vectors = all_sbert_vectors[val_df["item_indice"].values]
    val_item_features = np.hstack([val_item_features, val_sbert_vectors])

logger.info(f"{len(user_indices)=:,.0f}, {len(item_indices)=:,.0f}")

  0%|          | 0/5 [00:00<?, ?it/s]

Transforming chunks:   0%|          | 0/33 [00:00<?, ?it/s]

Transforming chunks:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-28 11:39:37.121 | INFO     | __main__:<module>:24 - len(user_indices)=19,578, len(item_indices)=4,630


# Train

In [21]:
rating_dataset = UserItemBinaryDFDataset(
    train_df,
    "user_indice",
    "item_indice",
    args.rating_col,
    args.timestamp_col,
    item_feature=train_item_features,
)
val_rating_dataset = UserItemBinaryDFDataset(
    val_df,
    "user_indice",
    "item_indice",
    args.rating_col,
    args.timestamp_col,
    item_feature=val_item_features,
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [22]:
n_items = len(item_indices)
n_users = len(user_indices)

model = init_model(
    n_users, n_items, args.embedding_dim, item_feature_size, args.dropout
)
model

Ranker(
  (item_embedding): Embedding(4631, 128, padding_idx=4630)
  (user_embedding): Embedding(19578, 128)
  (gru): GRU(128, 128, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (item_feature_tower): Sequential(
    (0): Linear(in_features=929, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (fc_rating): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

#### Predict before train

In [23]:
val_df = val_rating_dataset.df
val_df.sample(10)

,user_id,parent_asin,rating,timestamp,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,parent_asin_rating_avg_prev_rating_30d,...,item_indice,main_category,title,description,categories,price,user_rating_cnt_90d,user_rating_avg_prev_rating_90d,user_rating_list_10_recent_asin,item_sequence
1025,AGOKYUNVDXPVGKFK7EISFW6WKZQA,B09PNLW45Z,1.0,2021-12-03 17:34:12.409,2.0,4.50,0.0,NaN,0.0,NaN,...,4495,Video Games,SteelSeries Arctis Pro Wireless Gaming Headset...,[Arctis Pro Wireless features rock solid dual ...,"[Video Games, PlayStation 4, Accessories, Head...",434.51,1,NaN,"B004AYCNQ6,B004LLHFAW,B00IRHE892,B01MG8P418,B0...","[-1, -1, 1655, 1733, 2655, 3491, 3625, 4626, 3..."
344,AH7GYKBRDAWT2673HJKOXPIT57JQ,B00ANHBJBY,0.0,2022-01-26 22:31:23.759,0.0,NaN,0.0,NaN,0.0,NaN,...,2286,Video Games,Pro Commander for Wii U,[Take command of your favorite Wii U games wit...,[],None,1,NaN,"B0036EWMIK,B003OAE79S,B000N5Z2L4,B07YBX5FVF,B0...","[1426, 1526, 526, 4272, 758, 2356, 1406, 1487,..."
949,AGYJI6XSILTABA3ZWALFM6SMEVOQ,B005ZBNXMG,0.0,2022-04-22 22:32:31.047,1.0,5.00,0.0,NaN,0.0,NaN,...,1966,Video Games,Mario & Sonic at the London 2012 Olympic Games...,"[Product Description, Arriving on Nintendo’s n...","[Video Games, Legacy Systems, Nintendo Systems...",29.72,1,NaN,"B07SM7G9CN,B07G1SC6BW,B07L6MJ6LD,B07J3P1GJM,B0...","[-1, 4164, 3970, 4049, 4018, 4388, 3697, 3576,..."
1320,AHTLGOWRXF4PNX6DEUZXVZWBBVTQ,B075S5CYZR,1.0,2022-03-31 17:41:58.454,0.0,NaN,0.0,NaN,0.0,NaN,...,3784,Video Games,Dragon's Dogma: Dark Arisen - Standard Edition...,"[Set in a huge open world, Dragon's Dogma: Dar...","[Video Games, PlayStation 4, Games]",34.42,5,3.000000,"B01BF9X6LO,B07YBXFDYN,B07VRD1TT1,B01GW3H3U8,B0...","[3277, 4282, 4209, 3360, 4299, 3940, 4276, 415..."
1455,AEAIDF2BE3UEOTRCCON2NIVLUDBA,B08N7VJBH6,1.0,2022-01-18 03:07:48.175,2.0,5.00,0.0,NaN,0.0,NaN,...,4419,Video Games,Super Mario Maker 2 - Nintendo Switch,[Platform:Nintendo Switch | Edition:Standard M...,"[Video Games, Nintendo Switch, Games]",49.94,2,5.000000,"B087NNZZM8,B07X5CZXKH,B07QQ8N7LL,B07348JQ3L,B0...","[-1, -1, -1, 4343, 4254, 4140, 3707, 4368, 438..."
1357,AHCFZPQLIV7236UBXSGU3I5SMXBQ,B008M4BNQ6,0.0,2021-11-11 03:37:41.655,1.0,4.00,0.0,NaN,0.0,NaN,...,2174,Video Games,Turtle Beach - Ear Force XLa Gaming Headset - ...,"[Turtle Beach Ear Force XLa, view larger, The,...","[Video Games, Legacy Systems, Xbox Systems, Xb...",None,4,4.666667,"B07STWQ38X,B072V478NR,B076GXJNDZ,B0C5HQ6L7F,B0...","[4170, 3705, 3798, 4612, 3691, 3087, 3479, 372..."
1133,AEXLFLCDXVF6CJ6JVGZADT2Z3R2Q,B07982R9BZ,1.0,2022-02-27 20:40:17.200,1.0,5.00,0.0,NaN,0.0,NaN,...,3848,Video Games,"Skull & Co. Skin, CQC and FPS Thumb Grip Set J...",[3 different designs - to meet all gamersâ n...,"[Video Games, Nintendo Switch, Accessories, Ha...",10.99,1,NaN,"B003JBHG7C,B00BLDJ1T6,B005DONQWC,B0088TN73M,B0...","[-1, -1, -1, -1, 1483, 2350, 1904, 2123, 4542,..."
579,AE4QIOB3UZVH6UIF2GYTHHQ772GQ,B087LSSNG1,1.0,2021-09-27 13:13:32.839,6.0,4.50,1.0,5.0,1.0,5.0,...,4337,Video Games,Xenoblade Chronicles: Definitive Edition - Nin...,[Discover the origins of Shulk as he and his c...,"[Video Games, Nintendo Switch, Games]",54.98,1,NaN,"B001NJMMHG,B087STK7JZ,B01N3ASPNV,B07T5QKKVP,B0...","[-1, -1, -1, -1, -1, 1114, 4348, 3527, 4176, 4..."
719,AEMGKQWYD5AY4SQQWERK57RDE5JQ,B0BN942894,1.0,2021-10-01 02:55:25.812,20.0,4.65,1.0,5.0,0.0,NaN,...,4573,Computers,"BENGOO Stereo Pro Gaming Headset for PS4, PC, ...",[],"[Video Games, PC, Accessories, Headsets]",33.99,2,5.000000,"B00Z9TMH1W,B01MG6DORB,B01GY35W22,B001EYUSSA,B0...","[-1, -1, 3061, 3490, 3377, 958, 4611, 3388, 10..."
1816,AGDAPPCYV472FOUKDGAHZRW766GA,B000B69E9G,1.0,2022-04-07 05:49:55.659,1.0,5.00,0.0,NaN,0.0,NaN,...,404,Video Games,Resident Evil 4 - PlayStation 2,[In Resident Evil 4 you'll know a new type of ...,"[Video Games, Legacy Systems, PlayStation Syst...",None,2,1.000000,"B00GM5

In [24]:
user_id = val_df.sample(1)[args.user_col].values[0]
# user_id = "AH4AOFTTDPHPAFAAVFMAF25H2LIQ"
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
with pd.option_context("display.max_colwidth", None):
    display(test_df)

user_id parent_asin  rating  \
361   AGBP6HAY2K6QPNW3KENGNXURANWQ  B00OUSKMQA     0.0   
441   AGBP6HAY2K6QPNW3KENGNXURANWQ  B003IPCF02     0.0   
761   AGBP6HAY2K6QPNW3KENGNXURANWQ  B001QCWS9M     1.0   
1230  AGBP6HAY2K6QPNW3KENGNXURANWQ  B002BSC4ZS     1.0   
1638  AGBP6HAY2K6QPNW3KENGNXURANWQ  B0053BCO00     0.0   
1864  AGBP6HAY2K6QPNW3KENGNXURANWQ  B07X5X5KF9     1.0   

                   timestamp  parent_asin_rating_cnt_365d  \
361  2022-07-01 01:16:06.861                          9.0   
441  2022-05-15 19:20:40.786                          3.0   
761  2022-07-01 01:16:06.861                          0.0   
1230 2022-05-15 19:20:40.786                          1.0   
1638 2022-07-01 01:13:49.018                          2.0   
1864 2022-07-01 01:13:49.018                          0.0   

      parent_asin_rating_avg_prev_rating_365d  parent_asin_rating_cnt_90d  \
361                                  3.000000                         0.0   
441                                  3.666667                         1.0   
761                                       NaN                         0.0   
1230                                 4.000000                         0.0   
1638                                 5.000000                         1.0   
1864                                      NaN                         0.0   

      parent_asin_rating_avg_prev_rating_90d  parent_asin_rating_cnt_30d  \
361                                      NaN                         0.0   
441                                      1.0                         0.0   
761                                      NaN                         0.0   
1230                                     NaN                         0.0   
1638                                     5.0                         0.0   
1864                                     NaN                         0.0   

      parent_asin_rating_avg_prev_rating_30d  ...  item_indice  main_category  \
361                                      NaN  ...         2879      Computers   
441                                      NaN  ...         1482    Video Games   
761                                      NaN  ...         1131    Video Games   
1230                                     NaN  ...         1219    Video Games   
1638                                     NaN  ...         1875    Video Games   
1864                                     NaN  ...         4257    Video Games   

                                                        title  \
361   Corsair Vengeance 1400 Gaming Headset (CA-9011123-NA-Y)   
441                            Halo Reach - Legendary Edition   
761                                                Overlord 2   
1230                                         Metroid: Other M   
1638                                             Mario Kart 7   
1864  WB Games Middle Earth: Shadow of Mordor - Playstation 4   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [25]:
test_row = test_df.loc[lambda df: df[args.rating_col].gt(0)].iloc[0]
item_id = test_row[args.item_col]
item_sequence = test_row["item_sequence"]
row_idx = test_row.name
item_feature = val_item_features[row_idx]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)
user = torch.tensor([user_indice])
item_sequence = torch.tensor([item_sequence])
item_feature = torch.tensor([item_feature])
item = torch.tensor([item_indice])

model.eval()
model.predict(user, item_sequence, item_feature, item)

2024-10-28 11:39:37.502 | INFO     | __main__:<module>:6 - Test predicting before training with user_id = AGBP6HAY2K6QPNW3KENGNXURANWQ and parent_asin = B001QCWS9M
/var/folders/mq/xtxzhvlj4m97tb8lgwxcnsb40000gn/T/ipykernel_54361/2733117192.py:12: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)



tensor([[0.4522]], grad_fn=<SigmoidBackward0>)

#### Training loop

##### Overfit 1 batch

In [26]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, mode="min", verbose=False
)

model = init_model(n_users, n_items, args.embedding_dim, item_feature_size, dropout=0)
lit_model = LitRanker(
    model,
    learning_rate=args.learning_rate,
    l2_reg=0.0,
    log_dir=args.notebook_persist_dp,
)

log_dir = f"{args.notebook_persist_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=100,
    overfit_batches=1,
    callbacks=[early_stopping],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | Ranker | 3.4 M  | train
-----------------------------------------
3.4 M     Trainable params
0         Non-trainable params
3.4 M     Total params
13.533    Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:251: UserWarning:

You requested to overfit but enabled val dataloader shuffling. We are turning off the val dataloader shuffling for you.

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:251: UserWarning:

You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_conne

Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=100` reached.
2024-10-28 11:39:46.803 | INFO     | src.ranker.trainer:on_fit_end:158 - Logging classification metrics...
2024-10-28 11:40:03.683 | INFO     | __main__:<module>:28 - Logs available at /Users/dvq/frostmourne/recsys-mvp/notebooks/data/027-refit-with-features-and-title/logs/overfit/lightning_logs/version_0


In [27]:
%tensorboard --logdir $trainer.log_dir

##### Fit on all data

In [28]:
all_items_df = train_df.drop_duplicates(subset=["item_indice"])
all_items_indices = all_items_df["item_indice"].values
all_items_features = item_metadata_pipeline.transform(all_items_df).astype(np.float32)
logger.info(
    f"Mean std over categorical and numerical features: {all_items_features.std(axis=0).mean()}"
)
if args.use_sbert_features:
    all_sbert_vectors = ann_index.get_vector_by_ids(all_items_indices.tolist()).astype(
        np.float32
    )
    logger.info(f"Mean std over text features: {all_sbert_vectors.std(axis=0).mean()}")
    all_items_features = np.hstack([all_items_features, all_sbert_vectors])

2024-10-28 11:40:04.961 | INFO     | __main__:<module>:4 - Mean std over categorical and numerical features: 0.996731698513031


  0%|          | 0/47 [00:00<?, ?it/s]

2024-10-28 11:40:06.146 | INFO     | __main__:<module>:9 - Mean std over text features: 0.015866756439208984


In [29]:
# papermill_description=fit-model
early_stopping = EarlyStopping(
    monitor="val_loss", patience=args.early_stopping_patience, mode="min", verbose=False
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persist_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_loss",
    mode="min",
)

model = init_model(
    n_users,
    n_items,
    args.embedding_dim,
    item_feature_size,
    dropout=args.dropout,
    item_embedding=pretrained_item_embedding,
)
lit_model = LitRanker(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persist_dp,
    evaluate_ranking=True,
    idm=idm,
    all_items_indices=all_items_indices,
    all_items_features=all_items_features,
    args=args,
    neg_to_pos_ratio=args.neg_to_pos_ratio,
    checkpoint_callback=checkpoint_callback,
)

log_dir = f"{args.notebook_persist_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    max_epochs=args.max_epochs,
    callbacks=[early_stopping, checkpoint_callback],
    accelerator=args.device if args.device else "auto",
    logger=args._mlf_logger if args.log_to_mlflow else None,
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | Ranker | 3.4 M  | train
-----------------------------------------
3.4 M     Trainable params
0         Non-trainable params
3.4 M     Total params
13.533    Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.



Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

2024-10-28 11:50:49.513 | INFO     | src.ranker.trainer:on_fit_end:152 - Loading best model from /Users/dvq/frostmourne/recsys-mvp/notebooks/data/027-refit-with-features-and-title/checkpoints/best-checkpoint.ckpt...
2024-10-28 11:50:49.770 | INFO     | src.ranker.trainer:on_fit_end:158 - Logging classification metrics...
2024-10-28 11:50:50.357 | INFO     | src.ranker.trainer:on_fit_end:161 - Logging ranking metrics...


Generating recommendations:   0%|          | 0/177 [00:00<?, ?it/s]

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:64: RuntimeWarning:

invalid value encountered in divide

2024/10/28 11:51:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run 027-refit-with-features-and-title at: http://localhost:5002/#/experiments/3/runs/8ee774a1531241098fabc21d944af626.
2024/10/28 11:51:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5002/#/experiments/3.


In [30]:
logger.info(
    f"Test predicting after training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
model.eval()
model.predict(user, item_sequence, item_feature, item)

2024-10-28 11:51:31.248 | INFO     | __main__:<module>:1 - Test predicting after training with user_id = AGBP6HAY2K6QPNW3KENGNXURANWQ and parent_asin = B001QCWS9M


tensor([[0.3505]], grad_fn=<SigmoidBackward0>)

# Load best checkpoint

In [31]:
logger.info(f"Loading best checkpoint from {checkpoint_callback.best_model_path}...")
args.best_checkpoint_path = checkpoint_callback.best_model_path

best_trainer = LitRanker.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    model=init_model(
        n_users,
        n_items,
        args.embedding_dim,
        item_feature_size,
        dropout=0,
        item_embedding=pretrained_item_embedding,
    ),
)

2024-10-28 11:51:31.278 | INFO     | __main__:<module>:1 - Loading best checkpoint from /Users/dvq/frostmourne/recsys-mvp/notebooks/data/027-refit-with-features-and-title/checkpoints/best-checkpoint.ckpt...


In [32]:
best_model = best_trainer.model.to(lit_model.device)

In [33]:
best_model.eval()
best_model.predict(user, item_sequence, item_feature, item)

tensor([[0.3505]], grad_fn=<SigmoidBackward0>)

### Persist id mapping

In [34]:
if args.log_to_mlflow:
    # Persist id_mapping so that at inference we can predict based on item_ids (string) instead of item_index
    run_id = trainer.logger.run_id
    mlf_client = trainer.logger.experiment
    mlf_client.log_artifact(run_id, idm_fp)
    # Persist item_feature_metadata pipeline
    mlf_client.log_artifact(run_id, args.item_metadata_pipeline_fp)

### Wrap inference function and register best checkpoint as MLflow model

In [35]:
inferrer = RankerInferenceWrapper(best_model)

In [36]:
sample_input = {
    "user_ids": [idm.get_user_id(0)],
    "item_sequences": [[idm.get_item_id(0), idm.get_item_id(1)]],
    **{col: [train_df.iloc[0].fillna(0)[col]] for col in args.item_feature_cols},
    "item_ids": [idm.get_item_id(0)],
}
sample_output = inferrer.infer([0], [[0, 1]], [train_item_features[0]], [0])
sample_output

array([0.8046768], dtype=float32)

In [37]:
sample_input

{'user_ids': ['AE225O22SA7DLBOGOEIFL7FT5VYQ'],
 'item_sequences': [['0375869026', '9625990674']],
 'main_category': ['Video Games'],
 'categories': [array(['Video Games', 'Legacy Systems', 'PlayStation Systems',
         'PlayStation 3', 'Accessories', 'Controllers'], dtype=object)],
 'price': ['49.99'],
 'parent_asin_rating_cnt_365d': [76.0],
 'parent_asin_rating_avg_prev_rating_365d': [4.592105263157895],
 'parent_asin_rating_cnt_90d': [10.0],
 'parent_asin_rating_avg_prev_rating_90d': [4.3],
 'parent_asin_rating_cnt_30d': [3.0],
 'parent_asin_rating_avg_prev_rating_30d': [5.0],
 'parent_asin_rating_cnt_7d': [1.0],
 'parent_asin_rating_avg_prev_rating_7d': [5.0],
 'item_ids': ['0375869026']}

In [38]:
if args.log_to_mlflow:
    run_id = trainer.logger.run_id
    sample_output_np = sample_output
    signature = infer_signature(sample_input, sample_output_np)
    idm_filename = idm_fp.split("/")[-1]
    item_metadata_pipeline_filename = args.item_metadata_pipeline_fp.split("/")[-1]
    with mlflow.start_run(run_id=run_id):
        mlflow.pyfunc.log_model(
            python_model=inferrer,
            artifact_path="inferrer",
            artifacts={
                # We log the id_mapping to the predict function so that it can accept item_id and automatically convert ot item_indice for PyTorch model to use
                "idm": mlflow.get_artifact_uri(idm_filename),
                "item_metadata_pipeline": mlflow.get_artifact_uri(
                    item_metadata_pipeline_filename
                ),
            },
            model_config={"use_sbert_features": args.use_sbert_features},
            metadata={"use_sbert_features": args.use_sbert_features},
            signature=signature,
            input_example=sample_input,
            registered_model_name=args.mlf_model_name,
        )

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/mlflow/models/utils.py:523: FutureWarning:

Since MLflow 2.16.0, we no longer convert dictionary input example to pandas Dataframe, and directly save it as a json object. If the model expects a pandas DataFrame input instead, please pass the pandas DataFrame as input example directly.



Registered model 'ranker' already exists. Creating a new version of this model...
2024/10/28 11:51:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ranker, version 26
Created version '26' of model 'ranker'.


2024/10/28 11:51:40 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "user_ids": [
      "AE225O22SA7DLBOGOEIFL7FT5VYQ"
    ],
    "item_sequences": [
      [
        "0375869026",
        "9625990674"
      ]
    ],
    "main_category": [
      "Video Games"
    ],
    "categories": [
      [
        "Video Games",
        "Legacy Systems",
        "PlayStation Systems",
        "PlayStation 3",
        "Accessories",
        "Controllers"
      ]
    ],
    "price": [
      "49.99"
    ],
    "parent_asin_rating_cnt_365d": [
      76.0
    ],
    "parent_asin_rating_avg_prev_rating_365d": [
      4.592105263157895
    ],
    "parent_asin_rating_cnt_90d": [
      10.0
    ],
    "parent_asin_rating_avg_prev_rating_90d": [
      4.3
    ],
    "parent_asin_rating_cnt_30d": [
      3.0
    ],
    "parent_asin_rating_avg_prev_rating_30d": [
      5.0
    ],
    "parent_asin_rating_cnt_7d": [
      1.0
    ],
    "parent_asin_rating_avg_prev_rating

# Set the newly trained model as champion

In [39]:
if args.log_to_mlflow:
    val_roc_auc = trainer.logger.experiment.get_run(trainer.logger.run_id).data.metrics[
        "val_roc_auc"
    ]

    if val_roc_auc > args.min_roc_auc:
        logger.info(f"Aliasing the new model as champion...")
        model_version = (
            mlf_client.get_registered_model(args.mlf_model_name)
            .latest_versions[0]
            .version
        )

        mlf_client.set_registered_model_alias(
            name=args.mlf_model_name, alias="champion", version=model_version
        )

        mlf_client.set_model_version_tag(
            name=args.mlf_model_name,
            version=model_version,
            key="author",
            value="quy.dinh",
        )

2024-10-28 11:51:40.735 | INFO     | __main__:<module>:7 - Aliasing the new model as champion...


# Clean up

In [40]:
all_params = [args]

if args.log_to_mlflow:
    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.dict()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)

2024/10/28 11:51:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run 027-refit-with-features-and-title at: http://localhost:5002/#/experiments/3/runs/8ee774a1531241098fabc21d944af626.
2024/10/28 11:51:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5002/#/experiments/3.
